<a href="https://colab.research.google.com/github/qasimzee/isic2024/blob/main/isic_2024_challenge_table_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install scikit-learn==1.2.2
!pip install numpy==1.26.4

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
!ls "/content/drive/MyDrive/kaggle/isic-2024-data"

cnn.pkl			 sample_submission.csv	test-metadata.csv	  train-metadata.csv
logistic_regression.pkl  scaler.pkl		test-metadata.csv.gsheet  train-metadata-v1.csv
random_forest_model.pkl  test-image.hdf5	train-image.hdf5


In [23]:
METADATA_FILE = "/content/drive/MyDrive/kaggle/isic-2024-data/train-metadata.csv"
metadata_df = pd.read_csv(METADATA_FILE)


<ipython-input-23-6d834c4c6865>:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_df = pd.read_csv(METADATA_FILE)


In [24]:
train_df, test_df = train_test_split(metadata_df, test_size=0.05, random_state=42)

train_df


,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
102659,ISIC_2616636,0,IP_7059937,65.0,male,posterior torso,3.10,TBP tile: close-up,3D: XP,18.632880,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.995490
210006,ISIC_5281856,0,IP_8022574,50.0,male,posterior torso,4.90,TBP tile: close-up,3D: XP,20.476040,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.986030
121056,ISIC_3075395,0,IP_2248025,75.0,male,lower extremity,2.84,TBP tile: close-up,3D: white,20.409089,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.781312
195830,ISIC_4929825,0,IP_2859289,30.0,male,upper extremity,2.60,TBP tile: close-up,3D: XP,18.998953,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.995494
305494,ISIC_7640367,0,IP_4923309,70.0,male,upper extremity,2.60,TBP tile: close-up,3D: XP,18.773147,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,ISIC_6502755,0,IP_7965642,60.0,male,upper extremity,3.94,TBP tile: close-up,3D: XP,14.689290,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,94.829340
365838,ISIC_9129069,0,IP_2889176,45.0,female,posterior torso,2.74,TBP tile: close-up,3D: XP,24.387236,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,100.000000
131932,ISIC_3342513,0,IP_9577633,50.0,male,lower extremity,3.26,TBP tile: close-up,3D: XP,18.711190,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.980790
146867,ISIC_3720179,0,IP_3751225,75.0,male,anterior torso,2.60,TBP tile: close-up,3D: XP,16.110687,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.995995


In [25]:
dropped_columns = ["isic_id", "target", "patient_id", "lesion_id", "iddx_full", "iddx_1", "iddx_2", "iddx_3", "iddx_4", "iddx_5", "mel_mitotic_index", "mel_thick_mm", "tbp_lv_dnn_lesion_confidence"]
X = train_df.drop(columns=dropped_columns)
Y = train_df["target"]

# Replace NaN values
X['sex'] = X['sex'].replace(pd.NA, 'unknown')
X['anatom_site_general'] = X['anatom_site_general'].replace(pd.NA, 'unknown')



X.fillna(0, inplace=True)

In [26]:
# Identify columns with mixed types

for column in X.columns:
  if X[column].apply(type).nunique() > 1:
    print(f"{column}: {X[column].apply(type).unique()}")


In [27]:
# Identify categorical features
categorical_features = X.select_dtypes(include=['object']).columns

# Create a ColumnTransformer with OneHotEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)],
    remainder='passthrough')  # Pass through numerical columns unchanged


# Create a pipeline that first transforms the data, then fits the model
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('scaler', StandardScaler()),
                           ('model', LogisticRegression(class_weight='balanced'))])

# Fit the pipeline on the training data
pipeline.fit(X, Y)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  Index(['sex', 'anatom_site_general', 'image_type', 'tbp_tile_type',
       'tbp_lv_location', 'tbp_lv_location_simple', 'attribution',
       'copyright_license'],
      dtype='object'))])),
                ('scaler', StandardScaler()),
                ('model', LogisticRegression(class_weight='balanced'))])

In [28]:
test_df['sex'] = test_df['sex'].replace(pd.NA, 'unknown')
test_df['anatom_site_general'] = test_df['anatom_site_general'].replace(pd.NA, 'unknown')
test_df['lesion_id'] = test_df['lesion_id'].replace(pd.NA, 'unknown')
test_df['iddx_2'] = test_df['iddx_2'].replace(pd.NA, 'unknown')
test_df['iddx_3'] = test_df['iddx_3'].replace(pd.NA, 'unknown')
test_df['iddx_4'] = test_df['iddx_4'].replace(pd.NA, 'unknown')
test_df['mel_mitotic_index'] = test_df['mel_mitotic_index'].replace(pd.NA, 'unknown')

test_df.fillna(0, inplace=True)
# Identify columns with mixed types
for column in test_df.columns:
  if test_df[column].apply(type).nunique() > 1:
    print(f"{column}: {test_df[column].unique()}")


test_isic_ids = test_df['isic_id']
test_df = test_df.drop(columns=dropped_columns)


In [29]:
# Make predictions on the test data
y_pred_proba = pipeline.predict_proba(test_df)[:, 1]  # Probability of positive class (e.g., malignant)

# Create a DataFrame to map predictions back to 'isic_id'
predictions_df = pd.DataFrame({
    'isic_id': test_isic_ids,
    'target': y_pred_proba
})

# Print or save the predictions
# Optionally save to CSV
predictions_df

# # Create the submission DataFrame, using the saved 'isic_id'
# submission_df = pd.DataFrame({
#     'isic_id': test_isic_ids,
#     'target': test_predictions
# })

# # # Save to CSV
# submission_df #.to_csv('submission.csv', index=False)




,isic_id,target
278442,ISIC_6973879,0.115753
215021,ISIC_5407194,0.058694
209685,ISIC_5273739,0.422011
29648,ISIC_0802250,0.131960
323386,ISIC_8084953,0.641592
...,...,...
397655,ISIC_9915008,0.945602
273875,ISIC_6859044,0.013580
234128,ISIC_5885978,0.128661
336735,ISIC_8417947,0.642176


In [30]:
metadata_df[metadata_df['target'] == 1]['isic_id']

,isic_id
387,ISIC_0082829
935,ISIC_0096034
1245,ISIC_0104229
1846,ISIC_0119495
3478,ISIC_0157834
...,...
396165,ISIC_9877311
398671,ISIC_9941636
399936,ISIC_9972649
399943,ISIC_9972877


In [31]:
import pickle
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

# Assume you have trained your pipeline as described before

# Save the entire pipeline to a pickle file
with open('/content/drive/MyDrive/kaggle/isic-2024-data/logistic_regression.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

print("Model pipeline saved successfully!")


Model pipeline saved successfully!
